# Functions

## Standard Functions

Functions are used to give a name to repetitive code that needs to be used more than once with different parameters. If you ever find yourself copying and pasting code just to change a few variables, consider writing a function.

## Anonymous Functions

Sometimes, you may need a quick one line function to do something. The `lambda` keyword defines a callable function that isn't required to be given a name, hence, anonymous.

In [8]:
power = lambda b, e: b**e
print(power(2, 3))

8


Here, `power` is defined as a function that takes a base and an exponent and returns the result of exponentiation. However, didn't I say these were supposed to be anonymous? Let's design a more useful result. Let's say I have a list of numbers and I want to return the cube of all of them quickly. We can do this in one line by leveraging the `map()` function and a `lambda`. `map(func, iterable)` takes a function and applies it to each element in the provided `iterable` and returns a generator (which will be covered in the next section, so for now we will coerce it into a list).

In [9]:
cubes = list(map(lambda x: x**3, [1, 2, 3, 4,]))
print(cubes)

[1, 8, 27, 64]


`filter(func, iterable)` is another useful function that is most often paired with lambdas. It can filter an `iterable` based upon the output of `func`. If `func` returns `True`, the value is placed in the output, otherwise it is ignored. Let's say we only want numbers >=3.

In [10]:
min3 = list(filter(lambda x: x >= 3, [1, 2, 3, 4,]))
print(min3)

[3, 4]


## Generators

## Decorators

## Argument Unpacking